## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Aug. 19th Aug. 26th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [7]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
66,PAST SALE,2024-08-23,Condo/Co-op,901 Brickell Key Blvd #608,Miami,FL,33131.0,1300000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/901-Brickell-K...,MARMLS,A11593163,N,Y,25.768991,-80.186892
15,PAST SALE,2024-08-23,Condo/Co-op,13953 Kendale Lakes Cir Unit 201b,Miami,FL,33183.0,290000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/13953-Kendale-...,Beaches MLS,RX-11007255,N,Y,25.706342,-80.421354
32,PAST SALE,2024-08-23,Condo/Co-op,8024 Tatum Waterway Dr Unit 5B,Miami Beach,FL,33141.0,390000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/8024-Tat...,MARMLS,A11607160,N,Y,25.865557,-80.124887
41,PAST SALE,2024-08-23,Condo/Co-op,9711 Costa Del Sol Blvd Unit F-99,Doral,FL,33178.0,632500.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/9711-Costa-del...,MARMLS,A11592092,N,Y,25.806059,-80.355079
30,PAST SALE,2024-08-23,Condo/Co-op,600 Biltmore Way #518,Coral Gables,FL,33134.0,800000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/600-Bil...,Beaches MLS,RX-11001867,N,Y,25.747629,-80.267718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,PAST SALE,2024-08-19,Condo/Co-op,15300 SW 106th Ter #518,Miami,FL,33196.0,215000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15300-SW-106th...,MARMLS,A11592066,N,Y,25.667588,-80.443986
36,PAST SALE,2024-08-19,Condo/Co-op,19101 Mystic Pointe Dr #2811,Aventura,FL,33180.0,605000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19101-Mysti...,MARMLS,A11599252,N,Y,25.952775,-80.127636
64,PAST SALE,2024-08-19,Condo/Co-op,7757 SW 86th St Unit C-316,Miami,FL,33143.0,265000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/7757-SW-86th-S...,MARMLS,A11562183,N,Y,25.691529,-80.319893
28,PAST SALE,2024-08-19,Condo/Co-op,1900 N Bayshore Dr #2806,Miami,FL,33132.0,500000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1900-N-Bayshor...,MARMLS,A11616830,N,Y,25.795043,-80.187129


In [8]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Collect Agent Information

In [9]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [10]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [11]:
response_df = pd.DataFrame(response_list)

In [12]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [13]:
df_top_ten = merged_df

In [186]:
# columns_to_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','LOCATION',
#                    'LOT SIZE','DAYS ON MARKET',
#                   'HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','URL',
#                   'FAVORITE','INTERESTED','url','list_broker_email_1',
#                    'list_broker_email_2','buy_broker_email_1','buy_broker_email_2','latitude','longitude',
#                   'latest_price','num_full_baths','num_partial_baths','price_per_sq_ft','sq_ft','year_built',
#                   'assembled_address','data_source_name','mls_id']

# chart_df = merged_df.drop(columns=columns_to_drop)

# chart_df = chart_df.rename(columns={'SOLD DATE':'Close Date','ADDRESS':'Address','BEDS':'Beds','BATHS':'Baths',
#                          'PRICE':'Price','SQUARE FEET':'Square Feet','YEAR BUILT':'Year Built',
#                         '$/SQUARE FEET':'PSF','list_agent_name_1':'List Agent 1',
#                          'list_broker_name_1':'List Broker 1','SOURCE':'Source',
#                         'list_agent_name_2':'List Agent 2', 'list_broker_name_2':'List Broker 2',
#                         'buy_agent_name_1':'Buy Agent 1','buy_broker_name_1':'Buy Broker 1',
#                         'buy_agent_name_2':'Buy Agent 2','buy_broker_name_2':'Buy Broker 2'})

# merged_df['ADDRESS'] = merged_df['ADDRESS'] + ' ' + merged_df['CITY']

## Current Week's Values

In [14]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

114


In [15]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$851,894


In [16]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$542


In [17]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$97,115,930


In [14]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [18]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
110
Input Previous Week Condo Average Sales Price:
694298
Input Previous Week Condo Average PSF:
457
Input Previous Week Condo Sales total (ex: 198_000_000)
105000000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [19]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [20]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [21]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [22]:
pd.set_option('display.max_columns',None)

In [23]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-08-20,Condo/Co-op,1000 Biscayne Blvd #4202,Miami,FL,33132.0,7300000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1587.0,9939.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11589086,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Esther Percal,BHHS EWM Realty,Jason Zarco,BHHS EWM Realty,Christopher Bryan,EXP Realty LLC,None,None,1,orange


In [24]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [25]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-08-20,Condo/Co-op,1000 Biscayne Blvd #4202,Miami,FL,33132.0,7300000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1587.0,9939.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11589086,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Esther Percal,BHHS EWM Realty,Jason Zarco,BHHS EWM Realty,Christopher Bryan,EXP Realty LLC,None,None,1,orange
1,PAST SALE,2024-08-22,Condo/Co-op,2216 Fisher Island Dr #3106,Miami Beach,FL,33109.0,6119340.0,3.0,3.5,BAYSIDE VILLAGE CONDO,3011.0,NaN,1986.0,NaN,2032.0,9110.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2216-Fis...,MARMLS,A11598657,N,Y,25.758920,-80.143128,https://www.redfin.com/FL/Miami-Beach/2216-Fis...,Elena Bluntzer,One Sotheby's International Realty,None,None,Jill Eber,Coldwell Banker Realty,None,None,2,blue
2,PAST SALE,2024-08-23,Condo/Co-op,1455 Ocean Dr #1008,Miami Beach,FL,33139.0,6100000.0,2.0,2.0,IL VILLAGGIO SOUTH BEACH,2820.0,NaN,1998.0,NaN,2163.0,4416.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1455-Oce...,MARMLS,A11342643,N,Y,25.787342,-80.129359,https://www.redfin.com/FL/Miami-Beach/1455-Oce...,Nancy Batchelor,"Compass Florida, LLC.",None,None,Marine Lavaud,Douglas Elliman,None,None,3,blue
3,PAST SALE,2024-08-19,Condo/Co-op,100 S Pointe Dr #3205,Miami Beach,FL,33139.0,5650000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2003.0,NaN,3021.0,4416.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11590742,N,Y,25.767161,-80.132776,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Eddy Martinez,One Sotheby's International Re,Roland Ortiz,One Sotheby's International Re,Carla Ellek,"Westlake Realty Group, LLC.",None,None,4,blue
4,PAST SALE,2024-08-22,Condo/Co-op,5959 Collins Ave #907,Miami Beach,FL,33140.0,4300000.0,3.0,3.5,THE RESIDENCES AT THE BAT,2959.0,NaN,2005.0,NaN,1453.0,5486.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11600514,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Myroslava Mutsei,BrokerNation Real Estate,None,None,Ricky Alietti,Douglas Elliman,None,None,5,blue
5,PAST SALE,2024-08-21,Condo/Co-op,18555 Collins Ave #2405,Sunny Isles Beach,FL,33160.0,4000000.0,3.0,4.5,18555 COLLINS AVENUE COND,3171.0,NaN,2016.0,NaN,1261.0,6253.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11549774,N,Y,25.948195,-80.120410,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Jeannette Behrens,Berkshire Hathaway HomeServices Florida Realty,Marcela Parker,Berkshire Hathaway HomeServices Florida Realty,Michelle Judd,Dezer Platinum Realty LLC,None,None,6,blue
6,PAST SALE,2024-08-19,Condo/Co-op,17875 Collins Ave #602,Sunny Isles Beach,FL,33160.0,3400000.0,3.0,3.0,ACQUALINA OCEAN RESIDENCE,2214.0,NaN,2006.0,NaN,1536.0,4199.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11557442,N,Y,25.941257,-80.120772,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Yansy Checa,Douglas Elliman,None,None,Alejandro Vasquez,Million and Up Realty,None,None,7,blue
7,PAST SALE,2024-08-20,Condo/Co-op,1000 Brickell Plz Ph 5208,Miami,FL,33131.0,2800000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1912.0,NaN,2019.0,NaN,1464.0,2695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11483155,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Ralph Garcia,Lokation,None,None,Patricia Montalvan,Fortune Christie's International Real Estate,None,None,8,blue
8,PAST SALE,2024-08-19,Condo/Co-op,16001 Collins Ave #2206,Sunn

In [26]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [27]:
m.save('index.html')

## Data snagger

In [28]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-08-20,Condo/Co-op,1000 Biscayne Blvd #4202,Miami,FL,33132.0,7300000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1587.0,9939.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11589086,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Esther Percal,BHHS EWM Realty,Jason Zarco,BHHS EWM Realty,Christopher Bryan,EXP Realty LLC,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [30]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_08262024


# CREATE TEMPLATE 

In [31]:
muni_set = set(df_top_ten['CITY'])

In [32]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [33]:
df_top_ten.reset_index(inplace=True,drop=True)

In [34]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [35]:
top_sale

'ONE THOUSAND MUSEUM CONDO at 1000 Biscayne Blvd #4202 in Miami'

In [36]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [37]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [38]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: ONE THOUSAND MUSEUM CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,050,000M to $7,300,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Sunny Isles Beach, Miami, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 114 condo sales totaling $97,115,930 million from Aug. 19th to Aug. 26th. The previous week, brokers closed 110 condo sales totaling $105,000,000.

Last week’s units sold for an average of $851,894, higher than the $694,298 average price from the previous week. The average price per s

In [39]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [40]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [41]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [42]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: ONE THOUSAND MUSEUM CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,050,000M to $7,300,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Sunny Isles Beach, Miami, Miami Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 114 condo sales totaling $97,115,930 million from Aug. 19th to Aug. 26th. The previous week, brokers closed 110 condo sales totaling $105,000,000.

Last week’s units sold for an average of $851,894, higher than the $694,298 average price from the previous week. The average price per s

In [43]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami/1000-Biscayne-Blvd-33132/unit-4202/home/113531468


In [77]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/2216-Fisher-Island-Dr-33109/unit-3106/home/43103182


In [45]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/848-Brickell-Key-Dr-33131/unit-2606/home/43436487


In [46]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/16001-Collins-Ave-33160/unit-2206/home/43387989


In [47]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-3205/home/43372179


In [48]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/848-Brickell-Key-Dr-33131/unit-2606/home/43436487


In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-08-20,Condo/Co-op,1000 Biscayne Blvd #4202,Miami,FL,33132.0,7300000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1587.0,9939.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11589086,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Esther Percal,BHHS EWM Realty,Jason Zarco,BHHS EWM Realty,Christopher Bryan,EXP Realty LLC,None,None,1,orange,ONE THOUSAND MUSEUM CONDO at 1000 Biscayne Blv...
1,PAST SALE,2024-08-22,Condo/Co-op,2216 Fisher Island Dr #3106,Miami Beach,FL,33109.0,6119340.0,3.0,3.5,BAYSIDE VILLAGE CONDO,3011.0,NaN,1986.0,NaN,2032.0,9110.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2216-Fis...,MARMLS,A11598657,N,Y,25.758920,-80.143128,https://www.redfin.com/FL/Miami-Beach/2216-Fis...,Elena Bluntzer,One Sotheby's International Realty,None,None,Jill Eber,Coldwell Banker Realty,None,None,2,blue,BAYSIDE VILLAGE CONDO at 2216 Fisher Island Dr...
2,PAST SALE,2024-08-23,Condo/Co-op,1455 Ocean Dr #1008,Miami Beach,FL,33139.0,6100000.0,2.0,2.0,IL VILLAGGIO SOUTH BEACH,2820.0,NaN,1998.0,NaN,2163.0,4416.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1455-Oce...,MARMLS,A11342643,N,Y,25.787342,-80.129359,https://www.redfin.com/FL/Miami-Beach/1455-Oce...,Nancy Batchelor,"Compass Florida, LLC.",None,None,Marine Lavaud,Douglas Elliman,None,None,3,blue,IL VILLAGGIO SOUTH BEACH at 1455 Ocean Dr #100...
3,PAST SALE,2024-08-19,Condo/Co-op,100 S Pointe Dr #3205,Miami Beach,FL,33139.0,5650000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2003.0,NaN,3021.0,4416.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11590742,N,Y,25.767161,-80.132776,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Eddy Martinez,One Sotheby's International Re,Roland Ortiz,One Sotheby's International Re,Carla Ellek,"Westlake Realty Group, LLC.",None,None,4,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #3...
4,PAST SALE,2024-08-22,Condo/Co-op,5959 Collins Ave #907,Miami Beach,FL,33140.0,4300000.0,3.0,3.5,THE RESIDENCES AT THE BAT,2959.0,NaN,2005.0,NaN,1453.0,5486.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11600514,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Myroslava Mutsei,BrokerNation Real Estate,None,None,Ricky Alietti,Douglas Elliman,None,None,5,blue,THE RESIDENCES AT THE BAT at 5959 Collins Ave ...
5,PAST SALE,2024-08-21,Condo/Co-op,18555 Collins Ave #2405,Sunny Isles Beach,FL,33160.0,4000000.0,3.0,4.5,18555 COLLINS AVENUE COND,3171.0,NaN,2016.0,NaN,1261.0,6253.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11549774,N,Y,25.948195,-80.120410,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Jeannette Behrens,Berkshire Hathaway HomeServices Florida Realty,Marcela Parker,Berkshire Hathaway HomeServices Florida Realty,Michelle Judd,Dezer Platinum Realty LLC,None,None,6,blue,18555 COLLINS AVENUE COND at 18555 Collins Ave...
6,PAST SALE,2024-08-19,Condo/Co-op,17875 Collins Ave #602,Sunny Isles Beach,FL,33160.0,3400000.0,3.0,3.0,ACQUALINA OCEAN RESIDENCE,2214.0,NaN,2006.0,NaN,1536.0,4199.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11557442,N,Y,25.941257,-80.120772,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Yansy Checa,Douglas Elliman,None,None,Alejandro Vasquez,Million and Up Realty,None,None,7,blue,ACQUALINA OCEAN RESIDENCE at 17875 Collins Ave...
7,PAST SALE,2024-08-20,Condo/Co-op,1000 Brickell Plz Ph 5208,Miami,FL,33131.0,2800000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1912.0,

## Time on Market Calculator

In [75]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 6, 3) ## List (Earlier) date
date2 = datetime(2024, 8, 22) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

80


In [76]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-08-20,Condo/Co-op,1000 Biscayne Blvd #4202,Miami,FL,33132.0,7300000.0,4.0,5.5,ONE THOUSAND MUSEUM CONDO,4600.0,NaN,2019.0,NaN,1587.0,9939.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Biscayne-...,MARMLS,A11589086,N,Y,25.784171,-80.190078,https://www.redfin.com/FL/Miami/1000-Biscayne-...,Esther Percal,BHHS EWM Realty,Jason Zarco,BHHS EWM Realty,Christopher Bryan,EXP Realty LLC,None,None,1,orange,ONE THOUSAND MUSEUM CONDO at 1000 Biscayne Blv...
1,PAST SALE,2024-08-22,Condo/Co-op,2216 Fisher Island Dr #3106,Miami Beach,FL,33109.0,6119340.0,3.0,3.5,BAYSIDE VILLAGE CONDO,3011.0,NaN,1986.0,NaN,2032.0,9110.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2216-Fis...,MARMLS,A11598657,N,Y,25.758920,-80.143128,https://www.redfin.com/FL/Miami-Beach/2216-Fis...,Elena Bluntzer,One Sotheby's International Realty,None,None,Jill Eber,Coldwell Banker Realty,None,None,2,blue,BAYSIDE VILLAGE CONDO at 2216 Fisher Island Dr...
2,PAST SALE,2024-08-23,Condo/Co-op,1455 Ocean Dr #1008,Miami Beach,FL,33139.0,6100000.0,2.0,2.0,IL VILLAGGIO SOUTH BEACH,2820.0,NaN,1998.0,NaN,2163.0,4416.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1455-Oce...,MARMLS,A11342643,N,Y,25.787342,-80.129359,https://www.redfin.com/FL/Miami-Beach/1455-Oce...,Nancy Batchelor,"Compass Florida, LLC.",None,None,Marine Lavaud,Douglas Elliman,None,None,3,blue,IL VILLAGGIO SOUTH BEACH at 1455 Ocean Dr #100...
3,PAST SALE,2024-08-19,Condo/Co-op,100 S Pointe Dr #3205,Miami Beach,FL,33139.0,5650000.0,2.0,2.5,CONTINUUM ON SOUTH BEACH,1870.0,NaN,2003.0,NaN,3021.0,4416.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11590742,N,Y,25.767161,-80.132776,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,Eddy Martinez,One Sotheby's International Re,Roland Ortiz,One Sotheby's International Re,Carla Ellek,"Westlake Realty Group, LLC.",None,None,4,blue,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr #3...
4,PAST SALE,2024-08-22,Condo/Co-op,5959 Collins Ave #907,Miami Beach,FL,33140.0,4300000.0,3.0,3.5,THE RESIDENCES AT THE BAT,2959.0,NaN,2005.0,NaN,1453.0,5486.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11600514,N,Y,25.841880,-80.120969,https://www.redfin.com/FL/Miami-Beach/5959-Col...,Myroslava Mutsei,BrokerNation Real Estate,None,None,Ricky Alietti,Douglas Elliman,None,None,5,blue,THE RESIDENCES AT THE BAT at 5959 Collins Ave ...
5,PAST SALE,2024-08-21,Condo/Co-op,18555 Collins Ave #2405,Sunny Isles Beach,FL,33160.0,4000000.0,3.0,4.5,18555 COLLINS AVENUE COND,3171.0,NaN,2016.0,NaN,1261.0,6253.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11549774,N,Y,25.948195,-80.120410,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Jeannette Behrens,Berkshire Hathaway HomeServices Florida Realty,Marcela Parker,Berkshire Hathaway HomeServices Florida Realty,Michelle Judd,Dezer Platinum Realty LLC,None,None,6,blue,18555 COLLINS AVENUE COND at 18555 Collins Ave...
6,PAST SALE,2024-08-19,Condo/Co-op,17875 Collins Ave #602,Sunny Isles Beach,FL,33160.0,3400000.0,3.0,3.0,ACQUALINA OCEAN RESIDENCE,2214.0,NaN,2006.0,NaN,1536.0,4199.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11557442,N,Y,25.941257,-80.120772,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Yansy Checa,Douglas Elliman,None,None,Alejandro Vasquez,Million and Up Realty,None,None,7,blue,ACQUALINA OCEAN RESIDENCE at 17875 Collins Ave...
7,PAST SALE,2024-08-20,Condo/Co-op,1000 Brickell Plz Ph 5208,Miami,FL,33131.0,2800000.0,3.0,3.5,BRICKELL FLATIRON CONDO,1912.0,

## Clean CSV for Datawrapper Chart

In [51]:
chart_df = df_top_ten

In [52]:
chart_df = chart_df.fillna(" ")

In [53]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [54]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [55]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [56]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [57]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [63]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

datetime.datetime(2024, 8, 26, 11, 5, 22, 623786)

In [71]:
csv_date_string = today.strftime("%m_%d_%Y")

In [73]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")